# Bayesian Search
pytorch, skorch, optuna를 이용<br>
scikit-learn의 RandomSearch 등 역시 수행 가능

그외: target은 torch.LongTensor로 넣을 것 --> float가 아닌 int 형태

In [2]:
# data만 tensorflow를 이용해 load -> numpy 객체로 받아옴
import numpy as np
import tensorflow as tf
from tensorflow import keras

fashion_mnist = tf.keras.datasets.fashion_mnist.load_data()
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist
X_train, y_train = X_train_full[:-5000], y_train_full[:-5000]
X_valid, y_valid = X_train_full[-5000:], y_train_full[-5000:]

4422102/4422102 [==============================] - 0s 0us/step


# pytorch

In [14]:
X_train.shape

(55000, 28, 28)

In [139]:
import torch
from torch import nn
from torch.nn import functional as F

from skorch import NeuralNetClassifier

import optuna
from optuna.samplers import TPESampler
# maximized score를 목표로
study = optuna.create_study(direction='maximize', sampler=TPESampler(seed=0))

[I 2023-01-23 13:21:11,551] A new study created in memory with name: no-name-565b0015-6c40-4f43-b36b-da1200601506


In [140]:
class MLP_layers(nn.Module):
    def __init__(self, input_size, n_hidden, n_neurons):
        super().__init__()
        layers = [nn.Linear(input_size, n_neurons), nn.ReLU()]
        for _ in range(n_hidden-1):
            layers.append(nn.Linear(n_neurons,n_neurons))
            layers.append(nn.ReLU())
        layers.append(nn.Linear(n_neurons, 10))
        self.layers = nn.Sequential(*layers)
    def forward(self, x):
        # if len(x.shape) == 3:
        #     x = x.unsqueeze(1)
        x = torch.flatten(x,1)
        x = self.layers(x)
        return x

In [141]:
def objective(trial):
    # hyperparameter를 trial.suggest_로 정의
    n_hidden = trial.suggest_int('n_hidden', 0, 8)
    n_neurons = trial.suggest_int('n_neurons',16,256)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-2, log=True)
    optimizer = trial.suggest_categorical('optimizer', ['sgd', 'adam'])
    if str(optimizer) == 'sgd':
        optimizer = torch.optim.SGD
    elif str(optimizer) == 'adam':
        optimizer = torch.optim.Adam
    else:
        assert 'correct optimizer name is required'

    clf = NeuralNetClassifier(
        MLP_layers(28*28, n_hidden, n_neurons),
        max_epochs = 10,
        lr = learning_rate,
        optimizer = optimizer,
        criterion = nn.CrossEntropyLoss,
        device = 'cuda',
        train_split=None,
        verbose=0
    )

    # LongTensor는 int로 만들어준다.
    clf.fit(torch.Tensor(X_train), torch.Tensor(y_train).type(torch.LongTensor))
    score = clf.score(torch.Tensor(X_valid), torch.Tensor(y_valid).type(torch.LongTensor))
    return score

In [142]:
study.optimize(objective, n_trials=15)

[I 2023-01-23 13:21:24,921] Trial 0 finished with value: 0.8608 and parameters: {'n_hidden': 4, 'n_neurons': 188, 'learning_rate': 0.0016051911333587634, 'optimizer': 'sgd'}. Best is trial 0 with value: 0.8608.
[I 2023-01-23 13:21:37,037] Trial 1 finished with value: 0.8636 and parameters: {'n_hidden': 5, 'n_neurons': 121, 'learning_rate': 0.006074996073425695, 'optimizer': 'sgd'}. Best is trial 1 with value: 0.8636.
[I 2023-01-23 13:21:50,658] Trial 2 finished with value: 0.8316 and parameters: {'n_hidden': 7, 'n_neurons': 143, 'learning_rate': 0.001368009527972693, 'optimizer': 'sgd'}. Best is trial 1 with value: 0.8636.
[I 2023-01-23 13:22:00,552] Trial 3 finished with value: 0.292 and parameters: {'n_hidden': 0, 'n_neurons': 20, 'learning_rate': 0.0046263628414755864, 'optimizer': 'adam'}. Best is trial 1 with value: 0.8636.
[I 2023-01-23 13:22:15,024] Trial 4 finished with value: 0.7972 and parameters: {'n_hidden': 8, 'n_neurons': 208, 'learning_rate': 0.0008374496868436813, 'opti

In [143]:
print('best score : ',study.best_value)
print(study.best_params)

best score :  0.8858
{'n_hidden': 8, 'n_neurons': 255, 'learning_rate': 0.00010220736037059283, 'optimizer': 'adam'}
